# Subword Tokenizer
## : OOV(알 수 없는 문자) 문제를 해결
- 단어를 더 작은 단위인 subword로 분할하는 방법
- 단어 토큰 - 서브워드 토큰 - 문자 토큰
    - ex) birthplace = birth + place
 
1. BPE(Byte Pair Encoding) : 자주 나오는 문자 쌍을 병합하여 서브워드를 형성

    - 모든 문자를 개별적인 토큰으로 처리했을 때 가장 빈번히 나오는 문자 쌍을 병합

2. WordPiece : BERT 모델에서 사용되는 방법, 빈번한 서브워드를 찾는다.

3. SentencePiece : 문장 수준에서 토큰화한다.(단어의 경계를 무시)

    - 장점 : 메모리 감소, 희귀 단어(신조어 등), 다국어 지원


## 만약 aaabdaaabac라는 단어가 존재한다고 가정할 때 가장 많이 등장하는 알파벳을 탐색하고자 한다.
    1) aaabdaaabac (Z = aa)(치환)
    2) ZabdZabac (Y = ab)
    3) ZydZYac (X = ZY)
    4) XdXac

In [31]:
import re, collections
from IPython.display import display, Markdown, Latex

In [32]:
num_merges = 10

In [33]:
dictionary = {'1 o w </w>' : 5,
              'l o w e r </w>' : 2,
              'n e w e s t </w>' : 6,
              'w i d e s t </w>' : 3
             }

In [34]:
def get_stats(dictionary):
    # 인접한 문자 쌍의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items(): # 기본값이 0인 딕셔너리를 만든다.(키가 없을 경우 자동으로 0 반환)
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq

    print('pairs 빈도수 : ', dict(pairs))
    return pairs

In [35]:
def merge_dictionary(pair, v_in):
    # 인접한 문자 쌍(유니그램)을 병합해서 사전을 업데이트
    v_out = {}
    bigram = re.escape(' '.join(pair)) # 병합할 쌍을 공백으로 연결, escape : 특수문자 처리

    # \S -> 문자(다른 알파벳 사용 가능)
    # ! : 부정
    # (?<\S) : 문자 앞에 S(문자)가 등장하는지?
    # (?<!\S) 처리 : 공백이 아닌 문자 앞에 일치하지 않는다. (negative Lookbehind)
    # (?!\S) 처리 : 공백이 아닌 문자 뒤에 일치하지 않는다. (negative Lookahead)
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word) # 단어 병합
        v_out[w_out] = v_in[word] # 사전 추가

    return v_out

In [36]:
for i in range(num_merges):
    display(Markdown('### Iteration {}'.format(i+1)))
    pairs = get_stats(dictionary) 
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print('New merge : {}'.format(best))
    print('Dictionary : {}'.format(dictionary))

### Iteration 1

pairs 빈도수 :  {('1', 'o'): 5, ('o', 'w'): 7, ('w', '</w>'): 5, ('l', 'o'): 2, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
New merge : ('e', 's')
Dictionary : {'1 o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

pairs 빈도수 :  {('1', 'o'): 5, ('o', 'w'): 7, ('w', '</w>'): 5, ('l', 'o'): 2, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
New merge : ('es', 't')
Dictionary : {'1 o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

pairs 빈도수 :  {('1', 'ow'): 5, ('ow', '</w>'): 5, ('l', 'ow'): 2, ('ow', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge : ('1', 'ow')
Dictionary : {'1ow </w>': 5, 'l ow e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

pairs 빈도수 :  {('1ow', '</w>'): 5, ('l', 'ow'): 2, ('ow', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge : ('1ow', '</w>')
Dictionary : {'1ow</w>': 5, 'l ow e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

pairs 빈도수 :  {('l', 'ow'): 2, ('ow', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge : ('w', 'i')
Dictionary : {'1ow</w>': 5, 'l ow e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


In [37]:
print(bpe_codes)

{('wi', 'd'): 0, ('wid', 'est</w>'): 1, ('l', 'ow'): 2, ('low', 'e'): 3, ('lowe', 'r'): 4, ('lower', '</w>'): 5, ('e', 's'): 0, ('es', 't'): 1, ('est', '</w>'): 2, ('o', 'w'): 3, ('n', 'e'): 4, ('ne', 'w'): 5, ('new', 'est</w>'): 6, ('1', 'ow'): 7, ('1ow', '</w>'): 8, ('w', 'i'): 9}


In [46]:
# 새로운 단어가 등장할 때 대체할 방안
def get_pairs(word):
    paris = set()
    prev_char = word[0]
    for char in word[1:]:
        paris.add((prev_char, char))
        prev_char = char # 모든 순서쌍(문자쌍) set으로 반환
    return paris

In [52]:
def encode(orig):
    # 단어를 튜플로 변환하고 마지막에 </w> 추가(기존 작성했던 단어와 같이)
    word = tuple(orig) + ('</w>',) # 단어 tuple로(origin 전달)
    display(Markdown('__word split into characters: __ <tt>{}</tt>'.format(word)))

    pairs = get_pairs(word) # 처리한 모든 순서쌍 불러오기

    if not pairs: # 만약 get_pairs가 아무것도 반환이 되지 않았다면, 원래 단어를 반환한다.(순서쌍이 존재하지 않는다면)
        return orig # 원래 단어 반환

    iteration = 0 # 기본값 설정

    while True: # 무한반복
        iteration += 1 # 반복횟수 증가
        display(Markdown('__Iteration {}:__'.format(iteration)))

        print('Bigrams in the word: {}'.format(pairs)) # 현재 순서쌍이 어떤지 출력
        bigram = min(pairs, key=lambda pair: bpe_codes.get(pair, float('inf'))) # 빈도수가 가장 높은 순서쌍을 출력
        print('Candidate for merging: {}'.format(bigram))

        if bigram not in bpe_codes: # 더 이상 병합할 쌍이 없으면 '중지'
            display(Markdown('__Candidate not in BPE merges, algorithm stops.__'))
            break
            
        first, second = bigram # 밑 결과를 예시로 e와 s가 각각 first, second에 들어가 있다고 생각하면 된다.
        new_word = []
        i = 0

        while i < len(word): # 반복문
            try: # 예외처리
                j = word.index(first, i) # 첫 번째 문자의 인덱스를 찾는다.
                new_word.extend(word[i:j]) # i 이전까지 진행 후 j를 넣어준다.
                i = j
            except:
                new_word.extend(word[i:]) # 병합 후 new_word에 넣어주겠다.(es, t -> est)
                break # i부터 끝까지 진행 후 break를 통해 끝낸다.

            # 현재 문자와 다음 문자가 연결되어 있는지 확인
            if word[i] == first and i < len(word)-1 and word[i+1] == second: # 위 단어와 연관이 있는 경우
                new_word.append(first+second)
                i += 2 # 2씩 추가
            else: # 위와 연관이 없는 경우 
                new_word.append(word[i])
                i += 1 # 1씩 추가

        new_word = tuple(new_word) # 해당 단어를 튜플로 변환
        word = new_word # word로 바꿔준 뒤
        print('word after merging : {}'.format(word)) # word를 출력
        if len(word) == 1: # 만약 word의 길이가 1이라면,(더이상 진행할 것이 없기 때문에)
            break
        else:
            pairs = get_pairs(word)

    if word[-1] == '</w>': # 마지막 글자가 /w이라면,
        word = word[:-1] # 마지막 요소를 제거하고 출력
    elif word[-1].endswith('</w>'): # 마지막 글자가 /w가 아니라면,
        word = word[:-1] + (word[-1].replace('</w>', ''),) # 그대로 출력

    return word # 최종 word값 반환


In [53]:
# 겹치는 부분이 있을 경우 붙어서 결과가 출력된다.

In [48]:
encode('best')

__word split into characters: __ <tt>('b', 'e', 's', 't', '</w>')</tt>

__Iteration 1:__

Bigrams in the word: {('b', 'e'), ('e', 's'), ('s', 't'), ('t', '</w>')}
Candidate for merging: ('e', 's')
word after merging : ('b', 'es', 't', '</w>')


__Iteration 2:__

Bigrams in the word: {('b', 'es'), ('es', 't'), ('t', '</w>')}
Candidate for merging: ('es', 't')
word after merging : ('b', 'est', '</w>')


...

Bigrams in the word: {('est', '</w>'), ('b', 'est')}
Candidate for merging: ('est', '</w>')
word after merging : ('b', 'est</w>')


__Iteration 4:__

Bigrams in the word: {('b', 'est</w>')}
Candidate for merging: ('b', 'est</w>')


__Candidate not in BPE merges, algorithm stops.__

('b', 'est')

In [49]:
encode('lowing')

__word split into characters: __ <tt>('l', 'o', 'w', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

Bigrams in the word: {('o', 'w'), ('i', 'n'), ('g', '</w>'), ('w', 'i'), ('n', 'g'), ('l', 'o')}
Candidate for merging: ('o', 'w')
word after merging : ('l', 'ow', 'i', 'n', 'g', '</w>')


__Iteration 2:__

Bigrams in the word: {('i', 'n'), ('g', '</w>'), ('n', 'g'), ('l', 'ow'), ('ow', 'i')}
Candidate for merging: ('l', 'ow')
word after merging : ('low', 'i', 'n', 'g', '</w>')


__Iteration 3:__

Bigrams in the word: {('n', 'g'), ('g', '</w>'), ('i', 'n'), ('low', 'i')}
Candidate for merging: ('n', 'g')


__Candidate not in BPE merges, algorithm stops.__

('low', 'i', 'n', 'g')

In [51]:
encode('highinh')

__word split into characters: __ <tt>('h', 'i', 'g', 'h', 'i', 'n', 'h', '</w>')</tt>

__Iteration 1:__

Bigrams in the word: {('i', 'n'), ('n', 'h'), ('h', 'i'), ('h', '</w>'), ('i', 'g'), ('g', 'h')}
Candidate for merging: ('i', 'n')


__Candidate not in BPE merges, algorithm stops.__

('h', 'i', 'g', 'h', 'i', 'n', 'h')